# Code to process a 13D/F/G filing and pull out required fields

In [1]:
%pylab inline
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re

Populating the interactive namespace from numpy and matplotlib


## Do all extraction in one block of code - Run this

In [ ]:
%time

# Create an empty data frame. To be populated later with quarterly from info.
form_links = pd.DataFrame(columns = ['CIK', 'Company Name', 'Form Type', 'Date Filed', 'Filename'])

# Function to extract URLs of Form 13*. URLs to be used to access HTML/XML documents of interest
#
# Input: First and last years in range of years we want Form 13s pulled
#
# Output: Dataframe of all Form 13* filed during range of years specified.

def extract_13_url(first, last):

    for yr in range(first, last):
        for qr in range(1,5):

            #print(yr)
            df = pd.read_csv('MasterIndex'+str(yr)+ '_' + str(qr)+'.idx', encoding='latin1', sep = '\t')
            data = df.iloc[6:].reset_index(drop=True)
            data = data.rename(columns={'Description:           Master Index of EDGAR Dissemination Feed': 'messy'})
            meep = data.messy.str.split('|', expand = True)

            raw = meep.rename(columns={0: 'CIK', 1: 'Company Name', 2: 'Form Type', 3: 'Date Filed', 4: 'Filename'})
            #raw = raw[raw['Form Type'].str.contains('13D')| head['Form Type'].str.contains('13F') |\
            #          raw['Form Type'].str.contains('13G')].reset_index(drop=True)

            raw = raw[raw['Form Type'].str.match('.*13D.*|.*13F.*|.*13G.*')].reset_index(drop = True)

            form_links = form_links.append(raw)

    form_links = form_links.reset_index(drop = True)

In [ ]:
#extract_13_url(1997, 2019)

Save the dataframe to a csv file so we only need to load it down the pipeline.

In [ ]:
### DO NOT RUN ###
#form_links.to_csv('form_info.csv', index = False)
### DO NOT RUN ###

Below, test for the first filename to check for single file processing

In [2]:
df = pd.read_csv('form_info.csv', index_col = False)
df.head()

,CIK,Company Name,Form Type,Date Filed,Filename
0,1000014,RICE PARTNERS II L P,SC 13D,1997-03-14,edgar/data/1000014/0000930661-97-000604.txt
1,1000015,META GROUP INC,SC 13G/A,1997-01-30,edgar/data/1000015/0000941407-97-000032.txt
2,1000015,META GROUP INC,SC 13G/A,1997-02-12,edgar/data/1000015/0000950129-97-000571.txt
3,1000015,META GROUP INC,SC 13G,1997-02-12,edgar/data/1000015/0000080255-97-000155.txt
4,1000015,META GROUP INC,SC 13G,1997-02-13,edgar/data/1000015/0001000015-97-000003.txt


This is a very clean table. We can likely use the CIK from each row to match with the respective company's CIK in the Compustat table.

However, this Form 13* table includes both the target and acquirer's CIK for the same filing (in other words, we have duplicate filings). We will need to figure out how to workaround duplicates, or find another to uniquely identify each filing.

Fortunately, we know that each filing only contains the target's CUSIP number. Compustat data also contains a company's CUSIP number (which is unique). 

Therefore, we will have to extract the CUSIP no. from each filing and append it to the table as a new field.

In [ ]:
### DO NOT RUN ###
### Psedocode for later reference ###
if df['Date Filed'][1] >= last_filing & <= this_filing: 
    return that row's qtr
else:
    check the next row of datetbl 
### DO NOT RUN ###

#### Test - do not delete

In [ ]:
#Fields we want to collect
fields = ["COMPANY CONFORMED NAME","CENTRAL INDEX KEY","STANDARD INDUSTRIAL CLASSIFICATION","CUSIP NO."]

for x in range(1,10):
    url = 'https://www.sec.gov/Archives/' + df.Filename[x]
    
    f = requests.get(url)
    BeautifulSoup(f.content,'lxml').get_text()

### Content Extraction

In [4]:
td = df[df['Form Type'].str.match('.*13D')].reset_index(drop = True)

In [6]:
# Specify index of the file we want. Here, we will use index = 1 as a test.
x = 1

url = 'https://www.sec.gov/Archives/' + td.Filename[x]

In [7]:
# Fields we want to collect
fields = ["COMPANY CONFORMED NAME","CENTRAL INDEX KEY","STANDARD INDUSTRIAL CLASSIFICATION","CUSIP No."]

In [8]:
f = requests.get(url)
BeautifulSoup(f.content,'lxml').get_text()

'-----BEGIN PRIVACY-ENHANCED MESSAGE-----\nProc-Type: 2001,MIC-CLEAR\nOriginator-Name: webmaster@www.sec.gov\nOriginator-Key-Asymmetric:\n MFgwCgYEVQgBAQICAf8DSgAwRwJAW2sNKK9AVtBzYZmr6aGjlWyK3XmZv3dTINen\n TWSM7vrzLADbmYQaionwg5sDW3P6oaM5D3tdezXMm7z1T+B+twIDAQAB\nMIC-Info: RSA-MD5,RSA,\n Ja1MZAmdR+lxFmIP2eosVLh1cYQ/DGwYuDsfi7lUWUxXKgdvJT/+MKEDRnydF2AR\n qXjSP3zXtZwlYl1wQuJUMQ==\n\n0000950116-97-000432.txt : 19970310\n0000950116-97-000432.hdr.sgml : 19970310\nACCESSION NUMBER:\t\t0000950116-97-000432\nCONFORMED SUBMISSION TYPE:\tSC 13D\nPUBLIC DOCUMENT COUNT:\t\t1\nFILED AS OF DATE:\t\t19970307\nSROS:\t\t\tNONE\n\nSUBJECT COMPANY:\t\n\n\tCOMPANY DATA:\t\n\t\tCOMPANY CONFORMED NAME:\t\t\tSC&T INTERNATIONAL INC\n\t\tCENTRAL INDEX KEY:\t\t\t0001000079\n\t\tSTANDARD INDUSTRIAL CLASSIFICATION:\tCOMPUTER PERIPHERAL EQUIPMENT, NEC [3577]\n\t\tIRS NUMBER:\t\t\t\t860737579\n\t\tSTATE OF INCORPORATION:\t\t\tAZ\n\t\tFISCAL YEAR END:\t\t\t0630\n\n\tFILING VALUES:\n\t\tFORM TYPE:\t\tSC 13D\n\t\tSEC 

Code separately in the function below:
- Item 4
- Item 5
- (Date of Event Which Requires Filing of this Statement)
- SOURCE OF FUNDS*
- AGGREGATE AMOUNT BENEFICIALLY OWNED BY EACH REPORTING PERSON
- PERCENT OF CLASS REPRESENTED BY AMOUNT IN ROW (11)

In [15]:
# MAIN FUNCTIONS TO DO ALL EXTRACTION
#SEARCH LIST OF TEXT FOR ITEM LINE NUMBER 


def findLineNumber(list_of_text,text_item):
    res = [j for j in list_of_text if re.search(text_item,j)]
    for item in res:
        idx = list_of_text.index(item)
        
    return idx
        
def extract13D(url):
    print("URL: ",url)
    f = requests.get(url)
    text = BeautifulSoup(f.content,'lxml').get_text()
    #Split into lines
    list13d = text.splitlines()
    #Remove all stuff that starts with /xa0 and then all blank lines
    list13d = [j for j in list13d if j.startswith('\xa0')==False]
    list13d = [j for j in list13d if len(j)>0]
    for field in fields:
        print('Field: ',field)
        res = [j for j in list13d if re.search(field,j)]
        for r in res:
            x = re.split('[\t]',r) 
            y = [j for j in x if len(j)>0][-1]
            print(y)
        print(' ')
    #PROCESS ALL ITEMS
    #Item 4
    print('Field: Purpose of Transaction')
    start_idx = findLineNumber(list13d,'Item 4')
    end_idx = findLineNumber(list13d,'Item 5.')
    print(' '.join(list13d[(start_idx+1):end_idx]))


In [16]:
extract13D(url)

URL:  https://www.sec.gov/Archives/edgar/data/1000079/0000950116-97-000432.txt
Field:  COMPANY CONFORMED NAME
SC&T INTERNATIONAL INC
CAPITAL VENTURES INTERNATIONAL /E9/
 
Field:  CENTRAL INDEX KEY
0001000079
0001011712
 
Field:  STANDARD INDUSTRIAL CLASSIFICATION
COMPUTER PERIPHERAL EQUIPMENT, NEC [3577]
 []
 
Field:  CUSIP No.
CUSIP No.  783975 10 5                                       Page 2 of  5  Pages
CUSIP No.  783975 10 5                                       Page 3 of  5  Pages
CUSIP No.  783975 10 5                                       Page 4 of  5  Pages
CUSIP No.  783975 10 5                                       Page 5 of  5  Pages
 
Field: Purpose of Transaction


UnboundLocalError: local variable 'idx' referenced before assignment

Reqs:

* Pulled information should be in individual columns for each filing

Down the pipeline:

* Each filing can be matched to the target company's 10Q, w.r.t. the given quarter

Ideas:

* cumulative number of forms for each target/investor
* column to i.d. if initial form issue
* column to i.d. if amendement of form
* column to calc number (trading) days since last form issued for target

<b>
Reqs as of 5/16/2019:

* CUSIP from each 13D
* Date of filing
* binary column
</b>

Extract 13D only now

Fiscal year 

List of variables

Look at lit review for ideas

B/M ratio

firm size --> log(assets)

leverage

FCF

cash assets

R&D/assets

PPE assets

sales growth

profit margin

- institutional holdings
- corporate board structure

- control for corporate governance var (e.g. female bod)

Boardex variable descriptions

Risk metrics